In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [2]:
def plot_tweet_intensity(data,freq):
    data["timestamp"] = pd.to_datetime(data["timestamp"])
    tweets_per_hour = data.groupby([pd.Grouper(key='timestamp',freq=freq)]).size().reset_index(name='count')
    fig = px.line(tweets_per_hour, x="timestamp", y="count")
    fig.add_trace(go.Scatter(x=tweets_per_hour["timestamp"], y=tweets_per_hour["count"],
                        mode='markers',name='Tweet count'))
    fig.show()

In [206]:
def add_annotation(date, text, fig, anno_height, start_height = 0):
    fig.add_shape(
        # Line Vertical
        go.layout.Shape(
            type="line",
            x0=date,
            y0=start_height,
            x1=date,
            y1=anno_height,
            line=dict(
                color="RoyalBlue",
                width=3, 
                dash='dot'
            )
    ))
    
    fig.add_annotation(
        go.layout.Annotation(
            x=date,
            y=anno_height,
            xref="x",
            yref="y",
            text=text,
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-20,
            textangle=25, 
        )
    )

In [191]:
def annotate_chart_with_brexit_dates(fig, anno_height = 6000, start_height = 0):
    add_annotation('2018-12-17', 'Vote date on Brexit<br> deal announced', fig, anno_height, start_height)
    add_annotation('2019-02-15', 'May loses meaningful vote', fig, anno_height, start_height)
    add_annotation('2019-03-29', 'Brexit day I<br>May loses 2nd meaningful vote', fig, anno_height, start_height)
    add_annotation('2019-05-24', 'May announces resignation', fig, anno_height, start_height)
    add_annotation('2019-06-24', 'Boris Johnson becomes new PM', fig, anno_height, start_height)
    add_annotation('2019-10-17', 'Johnson agrees new Withdrawal<br> Agreement with EU', fig, anno_height, start_height)
    fig.update_annotations(dict(
            xref="x",
            yref="y",
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
    ))
    

In [202]:
def plot_tweet_sentiment_per_party(filename):
    data = pd.read_csv(filename)
    layout = go.Layout(yaxis=dict(range = [-1, 1]))

    fig = go.Figure(layout = layout)
    parties = data.party.unique()
    data['month_year'] = pd.to_datetime(data['timestamp']).dt.to_period('D')

    anno_height = 1.0

    for party in parties:
        df_party = data[data.party == party]
        counts = df_party.groupby('month_year')['score_num'].mean()
        labels = [str(c) for c in counts.index]
        fig.add_trace(go.Scatter(x=labels,y=counts.values,name=party))
    
    annotate_chart_with_brexit_dates(fig, anno_height, -1.0)

    fig.update_layout(
        xaxis_rangeslider_visible = True
    )
    fig.show()

In [221]:
def plot_party_sentiment(filename,freq, from_range, to_range):
    data = pd.read_csv(filename)
    layout = go.Layout(yaxis=dict(range = [from_range, to_range]))

    fig = go.Figure(layout = layout)
    parties = data.party.unique()
    
    anno_height = 450 if freq == 'D' else 6500
    
    data['month_year'] = pd.to_datetime(data['timestamp']).dt.to_period(freq)
    x_axis_labels = set(data['month_year'])

    for party in parties:
        df_party = data[data.party == party]
        counts = df_party['month_year'].value_counts().sort_index()
        labels = [str(c) for c in counts.index]
        fig.add_trace(go.Scatter(x=labels,y=counts.values,name=party))
    
  
    annotate_chart_with_brexit_dates(fig, anno_height)
    fig.update_layout(
        xaxis_rangeslider_visible = True
    )
    fig.show()

In [222]:
plot_party_sentiment("dataset/ppc_accounts_timeline_year_sentiment.csv",'M', 0, 6500)

In [223]:
plot_party_sentiment("dataset/ppc_accounts_timeline_year_sentiment.csv",'D', 0, 450)

In [224]:
plot_tweet_sentiment_per_party('preprocessed/ppc_accounts_timeline_year_sentiment_polarity.csv')